In [2]:
from deltalake import DeltaTable
from deltalake.writer import write_deltalake
from deltalake.exceptions import DeltaError
import os
import json
import pandas as pd
import duckdb
from dotenv import load_dotenv

# %% Carregar variáveis do .env
load_dotenv('C:\\Users\\Lincoln\\source\\repos\\lakehouse\\delta\\.env')


# ADLS storage account name and key 
account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")

storage_options = {
    "AZURE_STORAGE_ACCOUNT_NAME": os.getenv("AZURE_STORAGE_ACCOUNT_NAME"),
    "AZURE_STORAGE_ACCOUNT_KEY": os.getenv("AZURE_STORAGE_ACCOUNT_KEY"),
    "AZURE_STORAGE_CLIENT_ID": os.getenv("AZURE_STORAGE_CLIENT_ID"),
    "AZURE_STORAGE_TENANT_ID": os.getenv("AZURE_STORAGE_TENANT_ID"),
    "AZURE_STORAGE_ACCOUNT_SECRET": os.getenv("AZURE_STORAGE_ACCOUNT_SECRET")
    }



In [9]:
def escreve_delta(data: pd.DataFrame, path: str, mode: str = "append"):
    container = "bronze"
    path_delta = f'delta/delta-operacoes/{path}'
    uri = f'abfss://{container}@{account_name}.dfs.core.windows.net/{path_delta}'
    try:
        write_deltalake(uri, data, mode=mode, storage_options=storage_options)
        print(f"Dados escritos com sucesso na tabela Delta em: {uri}")
    except DeltaError as e:
        print(f"Erro ao escrever dados na tabela Delta: {e}")

In [22]:
def ler_delta(path: str) -> pd.DataFrame:
    container = "bronze"
    path_delta = f'delta/delta-operacoes/{path}'
    uri = f'abfss://{container}@{account_name}.dfs.core.windows.net/{path_delta}'
    try:
        dt = DeltaTable(uri, storage_options=storage_options)
        #df = dt.to_pandas()
        return dt
    except DeltaError as e:
        print(f"Erro ao ler dados da tabela Delta: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

In [5]:
con = duckdb.connect()
df = con.sql("SELECT * FROM 'C:\\Users\\Lincoln\\source\\repos\\lakehouse\\categories_test.csv'").to_df()
df

,category_id,category_name
0,1,Children Bicycles
1,2,Comfort Bicycles
2,3,Cruisers Bicycles
3,4,Cyclocross Bicycles
4,5,Electric Bikes
5,6,Mountain Bikes
6,7,Road Bikes


In [ ]:
escreve_delta(df, 'categories', mode='append')

In [25]:
dt = ler_delta('categories')
dt.to_pandas()

,category_id,category_name
0,2,Comfort Bicycles
1,3,Cruisers Bicycles
2,4,Cyclocross Bicycles
3,5,Electric Bikes
4,6,Mountain Bikes
5,7,Road Bikes
6,2,Comfort Bicycles
7,3,Cruisers Bicycles
8,4,Cyclocross Bicycles
9,5,Electric Bikes


In [26]:
#dt = DeltaTable(f'abfss://bronze@{account_name}.dfs.core.windows.net/delta/delta-operacoes/categories')
dt = ler_delta('categories')
#dt.to_pandas()
dt.delete('category_id == 7')

{'num_added_files': 1,
 'num_removed_files': 1,
 'num_deleted_rows': 2,
 'num_copied_rows': 10,
 'execution_time_ms': 1867,
 'scan_time_ms': 807,
 'rewrite_time_ms': 1059}

In [27]:
dt.version()

3

In [28]:
dt.to_pandas()

,category_id,category_name
0,2,Comfort Bicycles
1,3,Cruisers Bicycles
2,4,Cyclocross Bicycles
3,5,Electric Bikes
4,6,Mountain Bikes
5,2,Comfort Bicycles
6,3,Cruisers Bicycles
7,4,Cyclocross Bicycles
8,5,Electric Bikes
9,6,Mountain Bikes
